In [ ]:
# Propósito: Instalar y cargar librerías necesarias
!pip -q install yfinance --upgrade

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Propósito: Definir activos y descargar precios diarios
tickers = ["AAPL", "MSFT", "GOOGL"]  # Cambia, agrega o quita tickers (histórico de precios para empresa)
data = yf.download(tickers, period="6mo", interval="1d", auto_adjust=True, progress=False)

# Nos quedamos con el cierre para análisis básico
close = data["Close"].copy()
# Asegurar formato DataFrame aunque haya 1 ticker (Precio en la historia)
if isinstance(close, pd.Series):
    close = close.to_frame()

# Ver el precio de cierre
close.tail(10)

In [ ]:
# Resumen rápido de los datos descargados
resumen = pd.DataFrame({
    "ultimo_cierre": close.tail(1).T.iloc[:, 0],
    "fecha_ultimo": [close.index.max()] * close.shape[1],
    "fecha_primero": [close.index.min()] * close.shape[1],
})

# Vemos los datos
resumen

In [ ]:
# Calcular retornos diarios
returns = close.pct_change()

# Promedio diario para fines didácticos
avg_daily_return_pct = (returns.mean() * 100).round(3)
avg_daily_return_pct.to_frame("retorno_promedio_diario_%")

In [ ]:
# Propósito: Normalizar precios para comparar trayectorias
norm = (close / close.iloc[0]) * 100

plt.figure(figsize=(9, 5))
norm.plot(ax=plt.gca())
plt.title("Evolución comparada (precio normalizado = 100 al inicio)")
plt.xlabel("Fecha")
plt.ylabel("Índice (inicio=100)")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(7, 4))
avg_daily_return_pct.plot(kind="bar")
plt.title("Retorno promedio diario (%) en el periodo")
plt.xlabel("Ticker")
plt.ylabel("%")
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

In [ ]:
# --- Setup
import requests, pandas as pd

BASE = "https://api-colombia.com/api/v1"

def get_json(path, params=None):
    url = f"{BASE}/{path.strip('/')}"
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

# --- 1) Inspección rápida de recursos (ver qué trae cada endpoint)
country = get_json("country/Colombia")              # dict
departments = pd.DataFrame(get_json("Department"))  # list -> DataFrame
regions = pd.DataFrame(get_json("Region"))
airports = pd.DataFrame(get_json("Airport"))        # si es muy grande, filtra en memoria

print("Country keys:", list(country.keys()))
print("Departments columns:", departments.columns.tolist()[:10])
print("Regions columns:", regions.columns.tolist())
print("Airports columns:", airports.columns.tolist()[:10])

# --- 2) Búsqueda por nombre/keyword (útil para inputs de usuario)
# City por palabra clave (ej.: "Bogotá", "Medellín", "Cali")
cities_bogota = pd.DataFrame(get_json("city/search/Bogota"))
print("Ejemplo City search -> cols:", cities_bogota.columns.tolist())

# Department por nombre exacto (manejo de lista)
antioquia = get_json("Department/name/Antioquia")

if isinstance(antioquia, list) and len(antioquia) > 0:
    print("Total resultados:", len(antioquia))
    print("Keys del primer resultado:", list(antioquia[0].keys()))
    df_antioquia = pd.DataFrame(antioquia)
    print(df_antioquia[["id", "name", "population", "surface"]])
else:
    print("Sin resultados o formato inesperado")

# --- 3) Mini-análisis de negocio
# 3.1) Top departamentos por número de aeropuertos (proxy logístico)
if "departmentId" in airports.columns:
    top_dept_airports = (airports
                         .groupby("departmentId")
                         .size()
                         .reset_index(name="num_airports")
                         .sort_values("num_airports", ascending=False))
    print(top_dept_airports.head(10))
else:
    # Si no existe departmentId, imprime 5 filas para inspeccionar y ajustar columnas
    print(airports.head(5))

# 3.2) Atractivos turísticos más frecuentes por ciudad/departamento
# (revisar primero las columnas reales y ajustar los campos de agrupación)
tourism = pd.DataFrame(get_json("TouristicAttraction"))
print("Tourism columns:", tourism.columns.tolist())
# ejemplo genérico: intenta contar por una columna obvia si existe, p.ej. 'cityId' o 'departmentId'
group_cols = [c for c in ["cityId", "departmentId", "name"] if c in tourism.columns]
if group_cols:
    tourism_counts = (tourism.groupby(group_cols[:-1]).size()
                             .reset_index(name="num_attractions")
                             .sort_values("num_attractions", ascending=False))
    print(tourism_counts.head(10))

# --- 4) Visualización muy básica (bar chart simple)
# Objetivo docente: que el estudiante vea un ranking sin distracciones
# (si no tienes 'departmentId' en airports, cambia por cualquier columna válida)
if "departmentId" in airports.columns:
    top10 = top_dept_airports.head(10).set_index("departmentId")
    ax = top10["num_airports"].plot(kind="bar", title="Top 10 departamentos por número de aeropuertos")
    ax.set_xlabel("departmentId"); ax.set_ylabel("N.º aeropuertos")
